In [0]:
#notebooks/05_batch_scoring.py
# Databricks notebook source
# MAGIC %md
# MAGIC # Monitoramento de modelo de fraude

# COMMAND ----------

from pyspark.sql import functions as F

# Tabela de predições
PREDICTIONS_TABLE = "workspace.default.predictions_delta"

# COMMAND ----------

# Carregar previsões
pred_df = spark.table(PREDICTIONS_TABLE)

display(pred_df.limit(10))

# COMMAND ----------

# Distribuição das probabilidades de fraude
prob_stats = (
    pred_df
    .select("fraud_probability")
    .summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max")
)

display(prob_stats)

# COMMAND ----------

# Taxa de fraude prevista vs. real
fraud_rates = (
    pred_df
    .groupBy("date")
    .agg(
        F.avg("fraud_prediction").alias("predicted_fraud_rate"),
        F.avg("is_fraud").alias("actual_fraud_rate"),
        F.count("*").alias("total_transactions")
    )
    .orderBy("date")
)

display(fraud_rates)

# COMMAND ----------

# Diferença entre taxa prevista e real (erro de calibração)
fraud_rates = fraud_rates.withColumn(
    "calibration_error",
    F.col("predicted_fraud_rate") - F.col("actual_fraud_rate")
)

display(fraud_rates)

# COMMAND ----------

# Monitoramento por canal
channel_stats = (
    pred_df
    .groupBy("channel")
    .agg(
        F.avg("fraud_probability").alias("avg_fraud_probability"),
        F.avg("fraud_prediction").alias("predicted_fraud_rate"),
        F.avg("is_fraud").alias("actual_fraud_rate"),
        F.count("*").alias("total_transactions")
    )
    .orderBy("channel")
)

display(channel_stats)

# COMMAND ----------
# MAGIC %md
# MAGIC ### Métricas calculadas:
# MAGIC - Distribuição de `fraud_probability` (para detectar drift).
# MAGIC - Taxa de fraude prevista vs. real por dia.
# MAGIC - Erro de calibração (diferença entre previsto e observado).
# MAGIC - Estatísticas por canal de transação.

transaction_id,user_id,amount,country,channel,merchant_category,transaction_time,is_fraud,fraud_probability,fraud_prediction,date
228,5,34.91,PL,web,gaming,2024-06-12T08:18:55.000Z,0,0.0027767057831352042,0.0,2024-06-12
1073,22,64.8,IT,web,electronics,2024-06-12T06:39:16.000Z,0,0.0029279270909726737,0.0,2024-06-12
1706,34,64.72,ES,web,fashion,2024-06-12T07:45:10.000Z,0,0.003010121953600407,0.0,2024-06-12
1810,36,22.63,US,app,electronics,2024-06-12T07:14:01.000Z,0,0.0015123328650163148,0.0,2024-06-12
2039,40,102.35,US,web,electronics,2024-06-12T21:58:31.000Z,0,0.008971397495394249,0.0,2024-06-12
2501,49,342.23,TR,web,gaming,2024-06-12T03:45:55.000Z,0,0.0034091471550193138,0.0,2024-06-12
3429,68,202.11,PL,app,gaming,2024-06-12T04:06:44.000Z,0,0.0013629463981749401,0.0,2024-06-12
3520,70,108.58,TR,app,travel,2024-06-12T22:21:03.000Z,0,0.0015542302899316052,0.0,2024-06-12
3768,75,6.18,DE,web,gaming,2024-06-12T07:19:24.000Z,0,0.004790977405179295,0.0,2024-06-12
4476,90,237.72,FR,web,gaming,2024-06-12T21:13:34.000Z,0,0.0027965125710987418,0.0,2024-06-12


summary,fraud_probability
count,299695
mean,0.02214622144172635
stddev,0.10664913023128944
min,9.692613232004987E-4
25%,0.0015338192873107714
50%,0.002875103539904024
75%,0.0047392343470607804
max,0.9979570402427559


date,predicted_fraud_rate,actual_fraud_rate,total_transactions
2024-01-01,0.013013013013013013,0.01901901901901902,999
2024-01-02,0.011615628299894404,0.01689545934530095,947
2024-01-03,0.013793103448275862,0.020689655172413793,1015
2024-01-04,0.009155645981688708,0.019328585961342827,983
2024-01-05,0.018518518518518517,0.02263374485596708,972
2024-01-06,0.013402061855670102,0.02268041237113402,970
2024-01-07,0.013742071881606765,0.019027484143763214,946
2024-01-08,0.013631937682570594,0.02531645569620253,1027
2024-01-09,0.01680672268907563,0.023109243697478993,952
2024-01-10,0.01861427094105481,0.025853154084798345,967


date,predicted_fraud_rate,actual_fraud_rate,total_transactions,calibration_error
2024-01-01,0.013013013013013013,0.01901901901901902,999,-0.006006006006006008
2024-01-02,0.011615628299894404,0.01689545934530095,947,-0.005279831045406545
2024-01-03,0.013793103448275862,0.020689655172413793,1015,-0.006896551724137931
2024-01-04,0.009155645981688708,0.019328585961342827,983,-0.010172939979654119
2024-01-05,0.018518518518518517,0.02263374485596708,972,-0.004115226337448562
2024-01-06,0.013402061855670102,0.02268041237113402,970,-0.009278350515463918
2024-01-07,0.013742071881606765,0.019027484143763214,946,-0.005285412262156449
2024-01-08,0.013631937682570594,0.02531645569620253,1027,-0.011684518013631937
2024-01-09,0.01680672268907563,0.023109243697478993,952,-0.006302521008403363
2024-01-10,0.01861427094105481,0.025853154084798345,967,-0.007238883143743536


channel,avg_fraud_probability,predicted_fraud_rate,actual_fraud_rate,total_transactions
app,0.010166160909294588,0.004604357525988513,0.008042368226542527,147469
web,0.033751910001218056,0.023314019944030585,0.03564437086962805,152226
